# Modules

In [1]:
import csv
import numpy as np
import pandas as pd
from collections import Counter
from tqdm import tqdm

import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, Dataset

# Data preprocessing
0. Dataset 다운로드  
<br/>
1. Rating df 생성  
rating 데이터(train_ratings.csv)를 불러와 [user, item, rating]의 컬럼으로 구성된 데이터 프레임을 생성합니다.   
<br/>
2. Genre df 생성   
genre 정보가 담긴 데이터(genres.tsv)를 불러와 genre이름을 id로 변경하고, [item, genre]의 컬럼으로 구성된 데이터 프레임을 생성합니다.    
<br/>
3. Negative instances 생성   
rating 데이터는 implicit feedback data(rating :0/1)로, positive instances로 구성되어 있습니다. 따라서 rating이 없는 item중 negative instances를 뽑아서 데이터에 추가하게 됩니다.   
<br/>
4. Join dfs   
rating df와 genre df를 join하여 [user, item, rating, genre]의 컬럼으로 구성된 데이터 프레임을 생성합니다.   
<br/>
5. zero-based index로 mapping   
Embedding을 위해서 user,item,genre를 zero-based index로 mapping합니다.
    - user : 0-31359
    - item : 0-6806
    - genre : 0-17  
<br/>
6. feature matrix X, label tensor y 생성   
[user, item, genre] 3개의 field로 구성된 feature matrix를 생성합니다.   
<br/>
7. data loader 생성

## 데이터 다운로드
이곳에 대회 사이트(AI Stages)에 있는 data의 URL을 입력해주세요. 
- 데이터 URL은 변경될 수 있습니다.
- 예) `!wget https://aistages-prod-server-public.s3.amazonaws.com/app/Competitions/000176/data/data.tar.gz`

In [ ]:
# 0. Dataset 다운로드
!wget <대회 데이터 URL>
!tar -xf data.tar.gz

--16:03:12--  https://aistages-prod-server-public.s3.amazonaws.com/app/Competitions/000176/data/data.tar.gz
           => `data.tar.gz.1'
Resolving aistages-prod-server-public.s3.amazonaws.com... done.
Connecting to aistages-prod-server-public.s3.amazonaws.com[52.92.129.193]:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 33,424,489 [binary/octet-stream]

    0K .......... .......... .......... .......... ..........  0%  212.77 KB/s
   50K .......... .......... .......... .......... ..........  0%    1.14 MB/s
  100K .......... .......... .......... .......... ..........  0%  328.95 KB/s
  150K .......... .......... .......... .......... ..........  0%   24.41 MB/s
  200K .......... .......... .......... .......... ..........  0%  359.71 KB/s
  250K .......... .......... .......... .......... ..........  0%   24.41 MB/s
  300K .......... .......... .......... .......... ..........  1%    1.58 MB/s
  350K .......... .......... .......... .......... ..........  1

In [2]:
# 1. Rating df 생성
rating_data = "../../data/train/train_ratings.csv"

raw_rating_df = pd.read_csv(rating_data)
raw_rating_df
raw_rating_df['rating'] = 1.0 # implicit feedback
raw_rating_df.drop(['time'],axis=1,inplace=True)
print("Raw rating df")
print(raw_rating_df)

users = set(raw_rating_df.loc[:, 'user'])
items = set(raw_rating_df.loc[:, 'item'])

#2. Genre df 생성
genre_data = "../../data/train/genres.tsv"

raw_genre_df = pd.read_csv(genre_data, sep='\t')
raw_genre_df = raw_genre_df.drop_duplicates(subset=['item']) #item별 하나의 장르만 남도록 drop
# print(raw_genre_df)

genre_dict = {genre:i for i, genre in enumerate(set(raw_genre_df['genre']))}
raw_genre_df['genre']  = raw_genre_df['genre'].map(lambda x : genre_dict[x]) #genre id로 변경
print("Raw genre df - changed to id")
print(raw_genre_df)

Raw rating df
           user   item  rating
0            11   4643     1.0
1            11    170     1.0
2            11    531     1.0
3            11    616     1.0
4            11   2140     1.0
...         ...    ...     ...
5154466  138493  44022     1.0
5154467  138493   4958     1.0
5154468  138493  68319     1.0
5154469  138493  40819     1.0
5154470  138493  27311     1.0

[5154471 rows x 3 columns]
Raw genre df - changed to id
         item  genre
0         318      1
2        2571     12
5        2959     12
9         296     15
13        356     15
...       ...    ...
15925   73106     15
15926  109850     12
15929    8605     12
15931    3689     15
15932    8130      5

[6807 rows x 2 columns]


In [5]:
# 3. Negative instance 생성
print("Create Nagetive instances")
num_negative = 50
user_group_dfs = list(raw_rating_df.groupby('user')['item'])
first_row = True
user_neg_dfs = pd.DataFrame()

for u, u_items in tqdm(user_group_dfs):
    u_items = set(u_items)
    i_user_neg_item = np.random.choice(list(items - u_items), num_negative, replace=False)
    
    i_user_neg_df = pd.DataFrame({'user': [u]*num_negative, 'item': i_user_neg_item, 'rating': [0]*num_negative})
    if first_row == True:
        user_neg_dfs = i_user_neg_df
        first_row = False
    else:
        user_neg_dfs = pd.concat([user_neg_dfs, i_user_neg_df], axis = 0, sort=False)

raw_rating_df = pd.concat([raw_rating_df, user_neg_dfs], axis = 0, sort=False)

# 4. Join inter_df, genre_df dfs
joined_rating_df = pd.merge(raw_rating_df, raw_genre_df, left_on='item', right_on='item', how='inner')
# print("Joined rating df")
# print(joined_rating_df)

# 5. user, item을 zero-based index로 mapping
users = list(set(joined_rating_df.loc[:,'user']))
users.sort()
items =  list(set((joined_rating_df.loc[:, 'item'])))
items.sort()
genres =  list(set((joined_rating_df.loc[:, 'genre'])))
genres.sort()

# user 인덱싱
if len(users)-1 != max(users):
    users_dict = {users[i]: i for i in range(len(users))}
    joined_rating_df['user']  = joined_rating_df['user'].map(lambda x : users_dict[x])
    users = list(set(joined_rating_df.loc[:,'user']))
# item 인덱싱    
if len(items)-1 != max(items):
    items_dict = {items[i]: i for i in range(len(items))}
    joined_rating_df['item']  = joined_rating_df['item'].map(lambda x : items_dict[x])
    items =  list(set((joined_rating_df.loc[:, 'item'])))

joined_rating_df = joined_rating_df.sort_values(by=['user'])
joined_rating_df.reset_index(drop=True, inplace=True)

data = joined_rating_df
print("Data")
print(data)

n_data = len(data)
n_user = len(users)
n_item = len(items)
n_genre = len(genres)

print("# of data : {}\n# of users : {}\n# of items : {}\n# of genres : {}".format(n_data, n_user, n_item, n_genre))

Create Nagetive instances


100%|██████████| 31360/31360 [05:00<00:00, 104.49it/s]


Data
          user  item  rating  genre
0            0  2505     1.0     12
1            0  2659     1.0     15
2            0   191     1.0     12
3            0  3159     1.0      9
4            0   146     1.0      3
...        ...   ...     ...    ...
6722466  31359   833     1.0      1
6722467  31359  1197     1.0      3
6722468  31359  3760     0.0      3
6722469  31359  1269     0.0     15
6722470  31359   263     1.0     12

[6722471 rows x 4 columns]
# of data : 6722471
# of users : 31360
# of items : 6807
# of genres : 18


In [9]:
from sklearn.model_selection import train_test_split
from torch.utils.data import TensorDataset, DataLoader, Dataset

# fm/ffm에 사용할 데이터셋
X_train_context, X_test_context, y_train_context, y_test_context = train_test_split(
  data.drop(['rating'], axis=1), data['rating'], test_size=0.2, random_state=seed)

# PyTorch의 DataLoader에서 사용할 수 있도록 변환 
train_dataset_context = TensorDataset(torch.IntTensor(X_train_context.values), torch.IntTensor(y_train_context.values))
test_dataset_context = TensorDataset(torch.IntTensor(X_test_context.values), torch.IntTensor(y_test_context.values))

# DataLoader로 변환
train_loader = DataLoader(train_dataset_context, batch_size=batch_size, shuffle=data_shuffle)
test_loader = DataLoader(test_dataset_context, batch_size=batch_size, shuffle=False)

# FFM

## 모델

In [19]:
class FeaturesLinear(nn.Module):

    def __init__(self, field_dims: np.ndarray, output_dim: int=1):
        super().__init__()
        self.fc = torch.nn.Embedding(sum(field_dims), output_dim)
        self.bias = torch.nn.Parameter(torch.zeros((output_dim,)))
        self.offsets = np.array((0, *np.cumsum(field_dims)[:-1]), dtype=np.int64)
        
        # self.linear = nn.Linear(input_dim, 1, bias=True)
        #linear(x)
    def forward(self, x: torch.Tensor):
        """
        :param x: Long tensor of size ``(batch_size, num_fields)`` = (256,9)
        :return : (batch_size, output_dim=1)
        """
        x = x + x.new_tensor(self.offsets).unsqueeze(0) #[256,9]
        return torch.sum(self.fc(x), dim=1) + self.bias #self.fc(x) = (256,9,1) #self.bias = [0.] => [256,1]

In [13]:
class FieldAwareFactorizationMachine(nn.Module):

    def __init__(self, field_dims: np.ndarray, embed_dim: int):
        super().__init__()
        self.num_fields = len(field_dims)
        self.embeddings = torch.nn.ModuleList([
            torch.nn.Embedding(sum(field_dims), embed_dim) for _ in range(self.num_fields)
        ])
        self.offsets = np.array((0, *np.cumsum(field_dims)[:-1]), dtype=np.int64)
        for embedding in self.embeddings:
            torch.nn.init.xavier_uniform_(embedding.weight.data)

    def forward(self, x: torch.Tensor):
        """
        :param x: Long tensor of size ``(batch_size, num_fields)`` #[256,9]
        """
        x = x + x.new_tensor(self.offsets).unsqueeze(0)
        xs = [self.embeddings[i](x) for i in range(self.num_fields)]
        ix = list()
        for i in range(self.num_fields - 1):
            for j in range(i + 1, self.num_fields):
                ix.append(xs[j][:, i] * xs[i][:, j])
        ix = torch.stack(ix, dim=1)
        
        return ix #[256,36,8]

In [14]:
class FieldAwareFactorizationMachineModel(nn.Module):

    def __init__(self, field_dims: np.ndarray, embed_dim: int):
        super().__init__()
        self.linear = FeaturesLinear(field_dims)
        self.ffm = FieldAwareFactorizationMachine(field_dims, embed_dim)
        
    def forward(self, x: torch.Tensor):
        """
        :param x: Long tensor of size ``(batch_size, num_fields)``
        """
        ffm_term = torch.sum(torch.sum(self.ffm(x), dim=1), dim=1, keepdim=True)
        x = self.linear(x) + ffm_term
        return torch.sigmoid(x.squeeze(1))

## 평가메트릭 및 train, test 함수

In [10]:
from sklearn.metrics import mean_squared_error
def rmse(true: list, pred: list) -> float:
    return np.sqrt(mean_squared_error(true, pred))

In [24]:
def train(model: type, optimizer: torch.optim, data_loader: DataLoader, criterion: torch.nn, device: str, log_interval: int=100) -> None:
    model.train()
    total_loss = 0
    tk0 = tqdm(data_loader, smoothing=0, mininterval=1.0)
    for i, (fields, target) in enumerate(tk0):
        fields, target = fields.to(device), target.to(device)
        y = model(fields)
        loss = criterion(y, target.float())
        model.zero_grad()
        loss.backward()
        optimizer.step()
        total_loss += loss.item()
        if (i + 1) % log_interval == 0:
            tk0.set_postfix(loss=total_loss / log_interval)
            total_loss = 0

def test(model: type, data_loader: DataLoader, device: str) -> float:
    model.eval()
    targets, predicts = list(), list()
    with torch.no_grad():
        for fields, target in tqdm(data_loader, smoothing=0, mininterval=1.0):
            fields, target = fields.to(device), target.to(device)
            y = model(fields)
            targets.extend(target.tolist())
            predicts.extend(y.tolist())
    return rmse(targets, predicts)

## 하이퍼파라미터 설정 및 train/test

In [15]:
ffm_field_dims = np.array([len(users_dict), len(items_dict), len(genre_dict)]) #, dtype=np.uint32)

In [28]:
######## Hyperparameter ########
batch_size = 256 # 배치 사이즈
data_shuffle = True
embed_dim = 8 # embed feature의 dimension
epochs = 1 # epoch 돌릴 횟수
learning_rate = 0.001 # 학습이 반영되는 정도를 나타내는 파라미터
weight_decay=1e-6 # 정규화를 위한 파라미터
gpu_idx = 0

seed=42

torch.manual_seed(seed)
if torch.cuda.is_available():
    torch.cuda.manual_seed_all(seed)
    torch.backends.cudnn.deterministic = True

# device = torch.device("cuda:{}".format(gpu_idx) if torch.cuda.is_available() else "cpu")
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
# print("Capturing:", torch.cuda.is_current_stream_capturing())
print(device)

cuda


In [29]:
criterion = torch.nn.BCELoss()
model = FieldAwareFactorizationMachineModel(ffm_field_dims, embed_dim).to(device)
optimizer = torch.optim.Adam(params=model.parameters(), lr=learning_rate, amsgrad=True, weight_decay=weight_decay)

for epoch in range(epochs):
    train(model, optimizer, train_loader, criterion, device)
    auc_score = test(model, train_loader, device)
    print('epoch:', epoch, 'validation: rmse_score:', auc_score)

100%|██████████| 21008/21008 [00:50<00:00, 415.53it/s]


epoch: 0 validation: roc_auc_score: 0.2922893602566495


In [31]:
test(model, test_loader, device)

100%|██████████| 5252/5252 [00:11<00:00, 438.48it/s]


0.2977499089237386

# DeepFM

# Training

In [ ]:
device = torch.device('cuda')
input_dims = [n_user, n_item, n_genre]
embedding_dim = 10
model = DeepFM(input_dims, embedding_dim, mlp_dims=[30, 20, 10]).to(device)
bce_loss = nn.BCELoss() # Binary Cross Entropy loss
lr, num_epochs = 0.01, 10
optimizer = optim.Adam(model.parameters(), lr=lr)

for e in tqdm(range(num_epochs)) :
    for x, y in train_loader:
        x, y = x.to(device), y.to(device)
        model.train()
        optimizer.zero_grad()
        output = model(x)
        loss = bce_loss(output, y.float())
        loss.backward()
        optimizer.step()
        

100%|██████████████████████████████████████████████████████████████████████████████████| 10/10 [13:14<00:00, 79.43s/it]


# Evaluation
평가는 모델이 postive instance에 대해 0.5이상, negative instance에 대해 0.5미만의 값을 예측한 Accuracy를 측정하여 진행됩니다.

In [ ]:
correct_result_sum = 0
for x, y in test_loader:
    x, y = x.to(device), y.to(device)
    model.eval()
    output = model(x)
    result = torch.round(output)
    correct_result_sum += (result == y).sum().float()

acc = correct_result_sum/len(test_dataset)*100
print("Final Acc : {:.2f}%".format(acc.item()))

Final Acc : 90.43%


###**콘텐츠 라이선스**

<font color='red'><b>**WARNING**</b></font> : **본 교육 콘텐츠의 지식재산권은 재단법인 네이버커넥트에 귀속됩니다. 본 콘텐츠를 어떠한 경로로든 외부로 유출 및 수정하는 행위를 엄격히 금합니다.** 다만, 비영리적 교육 및 연구활동에 한정되어 사용할 수 있으나 재단의 허락을 받아야 합니다. 이를 위반하는 경우, 관련 법률에 따라 책임을 질 수 있습니다.

